#Import path and library

In [1]:
import pandas as pd
import numpy as np
import os
import json

In [2]:
p = '/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Vietstock/'
p1 = f'{p}Phong /TRANSLATE/'
path_lake = '/content/drive/MyDrive/Data Lake/'
dict_path = json.load(open(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /path.json'))
all_com = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_First (1_4).xlsx')['Symbol'])
all_com_2 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase2.xlsx')['Symbol'])
all_com_3 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase3.xlsx')['Symbol'])
all_com_4 = list(pd.read_excel(f'{path_lake}Ingestion/Day 0/Vietstock/Phong /List_Com_Phase4.xlsx')['Symbol'])
all_com = all_com+all_com_2+all_com_3+all_com_4
feature = pd.read_excel(f'{p}Phong /Compare_Feature.xlsx', sheet_name = 'Balance Sheet_ Vang + Phong ')
list_feature = feature[['Feature']][:58]

In [3]:
data_BS = pd.read_excel(f'{p1}BalanceSheet.xlsx',sheet_name='VietStock')
data_IS = pd.read_excel(f'{p1}IncomeStatement.xlsx',sheet_name='VietStock').rename(columns = {'Vietstock_Raw':'VIS_Raw'})
data_CF_Direct = pd.read_excel(f'{p1}CashFlowDirect.xlsx', sheet_name='VietStock')
data_CF_InDirect = pd.read_excel(f'{p1}CashFlowInDirect.xlsx', sheet_name='VietStock')

data_CF_InDirect['Vietstock_Raw_InDirect'] = data_CF_InDirect['Vietstock_Raw_InDirect'].str.upper()
data_CF_Direct['Vietstock_Raw_Direct'] = data_CF_Direct['Vietstock_Raw_Direct'].str.upper()
data_CF_Direct['Vietstock_Raw_Direct'].iloc[19] = 'LƯU CHUYỂN TIỀN THUẦN TỪ HOẠT ĐỘNG ĐẦU TƯ'
data_CF_InDirect = data_CF_InDirect.rename(columns = {'Vietstock_Raw_InDirect':'LƯU CHUYỂN TIỀN TỆ GIÁN TIẾP'})
data_CF_Direct = data_CF_Direct.rename(columns = {'Vietstock_Raw_Direct':'LƯU CHUYỂN TIỀN TỆ TRỰC TIẾP'})
data_CF = pd.concat([data_CF_InDirect[['Feature']], data_CF_Direct[['Feature']]]).reset_index(drop = True)

#F3: taok file F3 từ ngày lên sàn và xuống sàn

In [4]:
all_com_updown_exchange = pd.read_excel(f'{p}Phong /List_Com_DateUpDownExchange.xlsx')
all_com_updown_exchange.Up_Exchange_Quarter = pd.to_datetime(all_com_updown_exchange.Up_Exchange, format='%d/%m/%Y')
all_com_updown_exchange.Down_Exchange_Quarter = pd.to_datetime(all_com_updown_exchange.Down_Exchange, format='%d/%m/%Y')
all_com_updown_exchange['Up_Exchange_Quarter'] = pd.PeriodIndex(all_com_updown_exchange.Up_Exchange_Quarter, freq='Q').strftime(f'%q/%Y')
all_com_updown_exchange['Down_Exchange_Quarter'] = pd.PeriodIndex(all_com_updown_exchange.Down_Exchange_Quarter, freq='Q').strftime('%q/%Y')
all_com_updown_exchange['Up_Exchange'] = all_com_updown_exchange['Up_Exchange'].str[-4:]
all_com_updown_exchange['Down_Exchange'] = all_com_updown_exchange['Down_Exchange'].str[-4:]
all_com_updown_exchange['Up_Exchange'] = pd.to_numeric(all_com_updown_exchange['Up_Exchange'])
all_com_updown_exchange = all_com_updown_exchange.fillna(0)
all_com_updown_exchange['Up_Exchange'] = all_com_updown_exchange['Up_Exchange'].astype(int).astype(str)
# all_com_updown_exchange

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
list_time_quar = []
change_name_quy = {}
for year in range(2000, 2022):
    for i in range(1,5):
        list_time_quar.append(f'{i}/{year}')
        change_name_quy[f'Q{i}/{year}'] = f'{i}/{year}'
list_time_quar.append('1/2022')
list_time_quar.append('2/2022')
list_time_year = [str(i) for i in range(2000, 2023)]

In [6]:
def read_csv_and_merge(path):
    if 'CashFlow' in path:
        df_cafef = data_CF
    if 'BalanceSheet' in path:
        df_cafef = data_BS[['Feature']]
    if 'IncomeStatement' in path:
        df_cafef = data_IS[['Feature']]
    return df_cafef

In [ ]:
def fitter_updown(path, symbol):
    if os.path.exists(f'{path}{symbol}.csv'):
        df = pd.read_csv(f'{path}{symbol}.csv').rename(columns = {'Q1/2022': '1/2022'})
        # print(df)
    else: 
        df = read_csv_and_merge(path)
    if len(df) == 0 and len(df.columns) == 1:
        df = read_csv_and_merge(list_feature)
    row_symbol = all_com_updown_exchange[all_com_updown_exchange['Symbol'] == symbol]
    if 'Year' in path:
        up = row_symbol['Up_Exchange'].iloc[0]
        down = row_symbol['Down_Exchange'].iloc[0]
        if down == 0: down = list_time_year[len(list_time_year)-1]
        if up == str(0): up = str(2001)
        if up == str(2000): up = str(2001)
        # print(up, down)
        for ti in list_time_year[list_time_year.index(str(up))-1:list_time_year.index(str(down))+1]:
            if ti not in df.columns:
                # print(ti)
                df[ti] = np.nan
        df = df[['Feature']+ list_time_year[list_time_year.index(str(up))-1:list_time_year.index(str(down))+1]]
    if 'Quarter' in path:
        up = row_symbol['Up_Exchange_Quarter'].iloc[0]
        down = row_symbol['Down_Exchange_Quarter'].iloc[0]
        if down == 0: down = list_time_quar[len(list_time_quar)-1]
        if up == '2/2022' : return df[['Feature']]
        if up == 0: up = '2/2000'
        if up == '1/2000': up = '2/2000'
        print(up, down)
        print(list_time_quar.index(str(up))-1,list_time_quar.index(str(down))+1)
        for ti in list_time_quar[list_time_quar.index(str(up))-1:list_time_quar.index(str(down))+1]:
            # print(ti)
            if ti not in df.columns:
                df[ti] = np.nan
        # print(df)
        df = df[['Feature']+ list_time_quar[list_time_quar.index(str(up))-1:list_time_quar.index(str(down))+1]]

    try: first_date = list(df.columns).index(up)-1
    except: first_date = 1
    try: end_date = list(df.columns).index(down)
    except: end_date = len(list(df.columns)) -1
    # print(df)
    df1 = df.iloc[:, first_date:end_date]
    df1 = df1.rename(columns = change_name_quy)
    if first_date == 0:
        return df1
    else:
        df1.insert(0, 'Feature', df['Feature'])
    df1 = df1.fillna(str(0))
    return df1

symbol = 'HAP'
fitter_updown(dict_path['Vietstock_F2']['Year']['IS'], symbol)

In [ ]:
# pd.read_csv(dict_path['Vietstock_F2']['Year']['IS']+'BSP.csv')

In [9]:
for i in range(len(all_com)):
    symbol = all_com[i]
    print(i, symbol, end= ' ')
    for time in ['Year']:
        for finan in ['BS', 'IS']:
            path_in = dict_path['Vietstock_F2'][time][finan]
            path_out = dict_path['Vietstock_F3'][time][finan]
            fitter_updown(path_in, symbol).to_csv(path_out+f'{symbol}.csv', index = False)
            print(len(os.listdir(path_out)))
    # print('Done')

0 24H 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1992
1992
1 A32 1992
1992
2 AAA 1992
1992
3 AAM 1992
1992
4 AAT 1992
1992
5 AAV 1992
1992
6 ABC 1992
1992
7 ABG 1992
1992
8 ABR 1992
1992
9 ABS 1992
1992
10 ABT 1992
1992
11 AC4 1992
1992
12 ACC 1992
1992
13 ACE 1992
1992
14 ACG 1992
1992
15 ACL 1992
1992
16 ACM 1992
1992
17 ACS 1992
1992
18 ACV 1992
1992
19 ADC 1992
1992
20 ADG 1992
1992
21 ADP 1992
1992
22 ADS 1992
1992
23 AFC 1992
1992
24 AFX 1992
1992
25 AG1 1992
1992
26 AGB 1992
1992
27 AGC 1992
1992
28 AGD 1992
1992
29 AGE 1992
1992
30 AGF 1992
1992
31 AGG 1992
1992
32 AGM 1992
1992
33 AGP 1992
1992
34 AGX 1992
1992
35 ALC 1992
1992
36 ALP 1992
1992
37 ALS 1992
1992
38 ALT 1992
1992
39 ALV 1992
1992
40 AMC 1992
1992
41 AMD 1992
1992
42 AME 1992
1992
43 AMP 1992
1992
44 AMS 1992
1992
45 AMV 1992
1992
46 ANT 1992
1992
47 ANV 1992
1992
48 APC 1992
1992
49 APF 1992
1992
50 APH 1992
1992
51 API 1992
1992
52 APL 1992
1992
53 APP 1992
1992
54 APT 1992
1992
55 AQN 1992
1992
56 ARM 1992
1992
57 ASA 1992
1992
58 ASD 1992
1992
59 ASG 1992
1